In [1]:
import numpy as np
import random
from numpy.random import randint
from random import random as rnd

In [2]:
import datetime
from datetime import datetime
from datetime import time, timedelta

In [3]:
current_location = 'dhapakhel'

In [4]:
POIs = [current_location, "Pashupatinath Temple", "Boudhanath (Stupa)", "Swayambhunath Stupa", "Tribhuvan Museum","Dakshinkali Temple", "Narayanhiti Palace", "Garden of Dreams", "Kasthamandap", "Patan Durbar Square"]

In [5]:
POIs

['dhapakhel',
 'Pashupatinath Temple',
 'Boudhanath (Stupa)',
 'Swayambhunath Stupa',
 'Tribhuvan Museum',
 'Dakshinkali Temple',
 'Narayanhiti Palace',
 'Garden of Dreams',
 'Kasthamandap',
 'Patan Durbar Square']

In [6]:
def encode(POIs):
    encoded_POI = []
    for index, POI in enumerate(POIs):
        encoded_POI.append(index)
    return encoded_POI

In [7]:
encoded_POI = encode(POIs)

In [8]:
encoded_POI

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [9]:
class Individual:
    #Individual generation
    def generate_individual(self, POIs):
        constant_gene = POIs[0] #in every chromosomes starting location is the current location
        return [constant_gene]+random.sample(POIs[1:], len(POIs)-1)
    
    def population(self, population_size, POIs):
        return [self.generate_individual(POIs) for x in range(population_size)]
            

In [47]:
def Roulette_wheel_selection(fitness_score):
    Pc = [] #probability of selecting a chromosome
    sum_of_fitness = np.sum(fitness_score)
    for i in fitness_score:
        Pc.append(i/sum_of_fitness)
#     print(Pc)
#     print("Pc",Pc)
    return Pc

In [11]:
path_fitness_pair = {} #for global access
def dict_pair(population, fitness_score):
    for i in range(len(population)):
        path_fitness_pair[i] = fitness_score[i]
    return path_fitness_pair

In [12]:
print(path_fitness_pair)


{}


In [12]:
def rank_selected(selected_population):#fitness score of selected population
    key = []
    for i in selected_population:
        index = population.index(i)
        key.append(index)
    #fittest pairing => pairing the selected parents
    #sorting dict by value
    mylist = [path_fitness_pair[i] for i in key] #fitness score of selected parents
    # print(mylist)
    mylist.sort()#ascending order
    # print(mylist)
    mylist = mylist[::-1] # reverse ordering list , descending order
    # print(mylist)
    path_index = [list(path_fitness_pair.values()).index(i) for i in mylist]
    return path_index

In [13]:
from time_period import get_time

In [14]:
opening_time, closing_time = get_time(POIs[1:], 'Monday')

10:00:00
17:00:00
00:00:00
23:59:00
00:00:00
23:59:00
10:00:00
17:00:00
00:00:00
23:59:00
10:00:00
15:30:00
09:00:00
18:30:00
00:00:00
23:59:00
10:00:00
22:00:00


In [15]:
print(opening_time, closing_time)

[datetime.time(10, 0), datetime.time(0, 0), datetime.time(0, 0), datetime.time(10, 0), datetime.time(0, 0), datetime.time(10, 0), datetime.time(9, 0), datetime.time(0, 0), datetime.time(10, 0)] [datetime.time(17, 0), datetime.time(23, 59), datetime.time(23, 59), datetime.time(17, 0), datetime.time(23, 59), datetime.time(15, 30), datetime.time(18, 30), datetime.time(23, 59), datetime.time(22, 0)]


In [16]:
stay_time = 7200 # 2hrs stay in each poi

In [17]:
from get_time_matrix import get_time_matrix

In [18]:
distance_time = get_time_matrix(POIs)

In [19]:
distance_time

[[0, 1924, 2319, 2101, 1942, 2903, 2032, inf, 1782, 1128],
 [1924, 60, 660, 1620, 1260, 3240, 840, 780, 1380, 1260],
 [2319, 660, 60, 1920, 1620, 3600, 1140, 1080, 1620, 1680],
 [2101, 1620, 1860, 60, 1020, 2820, 1020, 960, 780, 1440],
 [1942, 1140, 1500, 1020, 60, 2580, 660, 540, 420, 1020],
 [2903, 3480, 3840, 2940, 2760, 60, 3060, 2940, 2640, 2520],
 [2032, 720, 1080, 1080, 840, 2940, 60, 180, 720, 1020],
 [inf, 720, 1080, 900, 720, 2820, 180, 60, 540, 960],
 [1782, 1320, 1680, 720, 420, 2460, 720, 600, 60, 960],
 [1128, 1260, 1620, 1380, 960, 2340, 1080, 900, 900, 60]]

In [20]:
def waiting_cost(starting_time,i, d ):
    if d == np.inf:
        d = 1000000;
#     if type(opening_time[i]) == 'str':
#         opening_time[i] = time(23, 59)
    arrival_time = (datetime.combine(datetime.today(),(starting_time))+ timedelta(seconds=d)).time()
#     print(opening_time[i])
    opening = datetime.combine(datetime.today(),(opening_time[i]))
    waiting_time = opening-(datetime.combine(datetime.today(),(starting_time))+ timedelta(seconds=d))
    next_time = (datetime.combine(datetime.today(),starting_time)+ timedelta(seconds=d))+timedelta(seconds = stay_time)
#     print(stay_time[i])
#     print(next_time.time())
    return max(waiting_time.total_seconds(), 0), next_time.time(), arrival_time

In [21]:
def delay_cost(leaving_time,i ):
    closing = datetime.combine(datetime.today(),(closing_time[i]))
    delay_time = datetime.combine(datetime.today(),(leaving_time))-closing
    return max(delay_time.total_seconds(), 0)

In [22]:
#Partially Mapped Crossover
def PMX(offspring1, offspring2, temp1, pivot_point1, pivot_point2):
    for i in range(len(offspring1)):
        if i < pivot_point1 or i >= pivot_point2: #change the points here
#             print('index',i)
            if temp1[i] not in offspring1:
                offspring1[i] = temp1[i]
#                 print(offspring1[i])
            else:
#                 print(temp1[i] in offspring1[2:4])
                while temp1[i] in offspring1[pivot_point1:pivot_point2]:
                    index = offspring1.index(temp1[i])
                    temp1[i] = offspring2[index]
                offspring1[i] = temp1[i]
#                 print(offspring1[i])

In [165]:
def two_opt(route, distance_matrix):
    best_route = route
    improved = True
    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 1, len(route)):
                if j - i == 1:
                    continue
                new_route = route[:]
                # Make sure route[i] and route[j] are integers representing indices
                new_route[i:j] = route[j - 1:i - 1:-1]  # perform 2-opt swap
                if calculate_route_distance(new_route, distance_matrix) < calculate_route_distance(best_route, distance_matrix):
                    best_route = new_route
                    improved = True
                    route = best_route
#         print(improved)
#     print(improved)
    return best_route

In [154]:
def calculate_route_distance(route, distance_matrix):
    distance = 0
    count = 0
    for i in route:
        if count>0:
            distance += distance_matrix[j][i]
        j =i
        count += 1
    return distance

In [232]:
def normalize_fitness(scores):
    min_score = min(scores)
    max_score = max(scores)
    normalized_scores = [(score - min_score) / (max_score - min_score) for score in scores]
    return normalized_scores

In [267]:
class GA:
    def __init__(self, population, crossover_rate):
        self.population = population
        self.crossover_rate = crossover_rate
        
    def fitness_evaluation(self, path):
        count = 0
        fitness_value = 0
        current_time = time(9,0)
        for i in path:
#             print('path',i)
            if count >= 1:
#                 cost, current_time, arrival_time = waiting_cost(current_time, i-1, distance_time[j][i]) #i-1 because opening and closing time of current_location is not mentioned
#                 if(current_time>time(18,0)):
#                     current_time = time(9,0)
#                 delay = delay_cost(current_time, i-1)
                fitness_value += distance_time[j][i] #+ cost + delay# distance consideration
    #             print(fitness_value)
#             print('fitness value', fitness_value/3600)
            j = i
            count += 1
            fitness_score = fitness_value/3600
        return 1/fitness_score
    
    #Choose a random number for making it act like a pointer in the wheel
    def selection(self, fitness_score):
        selected = []
       
        # calculating how many chromosomes to select for crossingover
        total_offspring = len(self.population) * self.crossover_rate
        num_parent_pairs = round(total_offspring / 2)
        num_selection = num_parent_pairs + 1
        
        for x in range(0, num_selection): 
            pointer = rnd()
            prob = 0
#             print(pointer)
            path_fitness_pair = dict_pair(self.population, fitness_score)
            for index, i in enumerate(Roulette_wheel_selection(fitness_score)):#roulette wheel
                prob += i #can take out cumulative sum instead see this............
                if prob > pointer:
#                     print(index)
                    selected.append(self.population[index])
#                     print('fitness_score',path_fitness_pair[index])
                    break
#             print(selected)
        return selected  
    
    #fittest pairing
    def pairing(self, selected_population):
        parents = [[selected_population[x],selected_population[x+1]] for x in range(len(selected_population)-1)]
        return parents
    
    #crossover
    #two point crossover
    def two_point_crossover(self, parent1, parent2):
        pivot_point1 = randint(1, len(parent1)-1)
#         print('pivot', pivot_point1)
        pivot_point2 = randint(1, len(parent1))
        while(pivot_point2 <= pivot_point1):
            pivot_point2 = randint(1, len(parent1))
    #     print(pivot_point1, pivot_point2)
    #     print(parent1, parent2)
        if random.random() < self.crossover_rate: 

            offspring1 = [-1]*len(parent1)
            offspring2 = [-1]*len(parent1)
            offspring1[pivot_point1:pivot_point2] = parent2[pivot_point1:pivot_point2]
            offspring2[pivot_point1:pivot_point2] = parent1[pivot_point1:pivot_point2]

        #     print(parent1, parent2)
            temp1 = parent1.copy()
            temp2 = parent2.copy() #copy garena bhane parent ma ni modification aauxa but why???

        #     print(offspring1)
            PMX(offspring1, offspring2, temp1, pivot_point1, pivot_point2)
            PMX(offspring2, offspring1, temp2, pivot_point1, pivot_point2)

            return [offspring1, offspring2]
        
        else:
            return [parent1, parent2]
    #     return offspring1, offspring2 =>list of tuples of list
    #     print(parent1, parent2)
    
    #Scramble mutation : choose two points then bichko genes lai shuffle garne
    def individual_for_mutation(self, mutation_rate = 0.01):
        individual_to_mutate = []
        for x in range(round(len(self.population)*mutation_rate)):
            individual_to_mutate.append(self.population[randint(0, len(self.population)-1)])
        return individual_to_mutate
    
    def scramble_mutation(self, individual):
        p1 = randint(1, len(individual)-1)
        p2 = randint(1, len(individual))
        while p1 >= p2:
            p2 = randint(1, len(individual))
        c2 = individual[p1:p2]
        random.shuffle(c2)
        for i in c2:
            individual[p1] = i
            p1 +=1
        return individual

In [262]:
def best_solution(population, fitness_score):
    best_fittest = np.max(fitness_score)
    index = fitness_score.index(best_fittest)
    best_individual = population[index]
    return best_fittest, best_individual

In [263]:
i = Individual()
population_size = 100
initial_population = i.population(population_size, encoded_POI)

In [274]:
population = initial_population

In [275]:
population

[[0, 4, 3, 6, 1, 5, 2, 8, 9, 7],
 [0, 7, 9, 8, 6, 5, 4, 1, 2, 3],
 [0, 2, 4, 7, 6, 3, 5, 9, 1, 8],
 [0, 2, 7, 5, 4, 6, 1, 8, 9, 3],
 [0, 8, 4, 2, 6, 5, 3, 7, 9, 1],
 [0, 5, 8, 6, 4, 9, 3, 7, 1, 2],
 [0, 4, 5, 6, 2, 8, 1, 9, 7, 3],
 [0, 5, 1, 2, 6, 8, 9, 7, 3, 4],
 [0, 5, 4, 7, 6, 1, 3, 2, 8, 9],
 [0, 7, 9, 5, 6, 4, 3, 8, 1, 2],
 [0, 1, 7, 6, 4, 5, 2, 9, 3, 8],
 [0, 4, 8, 2, 6, 5, 1, 9, 7, 3],
 [0, 5, 9, 1, 8, 3, 6, 2, 7, 4],
 [0, 7, 3, 6, 1, 4, 5, 8, 2, 9],
 [0, 8, 2, 3, 7, 4, 5, 9, 1, 6],
 [0, 8, 6, 4, 1, 7, 9, 5, 3, 2],
 [0, 6, 3, 8, 2, 9, 4, 5, 1, 7],
 [0, 6, 9, 1, 4, 5, 2, 7, 3, 8],
 [0, 7, 5, 8, 2, 9, 4, 6, 3, 1],
 [0, 3, 5, 7, 9, 2, 4, 1, 8, 6],
 [0, 4, 7, 1, 5, 2, 6, 9, 3, 8],
 [0, 1, 2, 7, 6, 4, 3, 5, 9, 8],
 [0, 4, 3, 5, 2, 7, 6, 1, 8, 9],
 [0, 5, 7, 9, 6, 1, 8, 2, 4, 3],
 [0, 2, 8, 6, 5, 4, 9, 3, 7, 1],
 [0, 9, 3, 6, 7, 4, 5, 2, 8, 1],
 [0, 8, 6, 2, 1, 7, 3, 5, 4, 9],
 [0, 6, 2, 7, 9, 8, 5, 4, 1, 3],
 [0, 2, 9, 5, 3, 7, 8, 6, 1, 4],
 [0, 1, 4, 6, 8, 9, 2, 3, 5, 7],
 [0, 7, 1,

In [279]:
def run(population):
    g = GA(population, 0.6)
    fitness_score = [g.fitness_evaluation(individual) for individual in population]
#     print(fitness_score)
    best_fittest, best_individual = best_solution(population, fitness_score)
    selected = g.selection(fitness_score)
#     print(selected)
    path_index = rank_selected(selected)
#     print(path_index)
    paired_parents = g.pairing(selected)
#     print(paired_parents)
    offsprings = []
    for i in paired_parents:
        offsprings.append(g.two_point_crossover(i[0], i[1]))
#     print(offsprings, len(offsprings))
    next_population = []
    nextpopulation = []
    for i in offsprings:
        next_population.extend(i)
#     print('nextpopulation',next_population)
    individual_to_mutate = g.individual_for_mutation()
#     print('individual_to_mutate',individual_to_mutate)
    for x in individual_to_mutate:
        next_population.append(g.scramble_mutation(x))
#     for x in next_population:
# #         print('route', x)
#         best_route = two_opt(x, distance_time)
# #         print('best_route',best_route)
#         nextpopulation.append(best_route)
# #     print('nextpopulation',next_population)
#     next_population = nextpopulation


#     initial_population = next_population
    return best_fittest, best_individual, next_population

In [277]:
def run(population):
    g = GA(population, 0.6)
    fitness_score = [g.fitness_evaluation(individual) for individual in population]
#     print(fitness_score)
    normalized_score = normalize_fitness(fitness_score)
#     print(normalized_score)
    best_fittest, best_individual = best_solution(population, normalized_score)
    selected = g.selection(normalized_score)
#     print(selected)
    path_index = rank_selected(selected)
#     print(path_index)
    paired_parents = g.pairing(selected)
#     print(paired_parents)
    offsprings = []
    for i in paired_parents:
        offsprings.append(g.two_point_crossover(i[0], i[1]))
#     print(offsprings, len(offsprings))
    next_population = []
    nextpopulation = []
    for i in offsprings:
        next_population.extend(i)
#     print('nextpopulation',next_population)
    individual_to_mutate = g.individual_for_mutation()
#     print('individual_to_mutate',individual_to_mutate)
    for x in individual_to_mutate:
        next_population.append(g.scramble_mutation(x))
#     for x in next_population:
# #         print('route', x)
#         best_route = two_opt(x, distance_time)
# #         print('best_route',best_route)
#         nextpopulation.append(best_route)
# #     print('nextpopulation',next_population)
#     next_population = nextpopulation


#     initial_population = next_population
    return best_fittest, best_individual, next_population

In [280]:
generation = 100
for i in range(100):
#     print(population)
    best_fittest, best_individual, next_population = run(population)
    print('Generation',i+1,':', 'Best Fittest:',best_fittest," Best Individual:",best_individual)
    while len(next_population) < population_size:
#         print(len(next_population))
#         print(population[randint(0, len(population))])
        next_population.append(population[randint(0, len(population))])
    print(len(next_population))
    population = next_population
    
route = []
for i in best_individual:
    route.append(POIs[i])
print(route)

Generation 1 : Best Fittest: 0.3851091142490372  Best Individual: [0, 9, 4, 7, 6, 1, 2, 8, 3, 5]
100
Generation 2 : Best Fittest: 0.3851091142490372  Best Individual: [0, 9, 4, 7, 6, 1, 2, 8, 3, 5]
100
Generation 3 : Best Fittest: 0.3851091142490372  Best Individual: [0, 9, 4, 7, 6, 1, 2, 8, 3, 5]
100
Generation 4 : Best Fittest: 0.3851091142490372  Best Individual: [0, 9, 4, 7, 6, 1, 2, 8, 3, 5]
100
Generation 5 : Best Fittest: 0.3851091142490372  Best Individual: [0, 9, 4, 7, 6, 1, 2, 8, 3, 5]
100
Generation 6 : Best Fittest: 0.3851091142490372  Best Individual: [0, 9, 4, 7, 6, 1, 2, 8, 3, 5]
100
Generation 7 : Best Fittest: 0.3851091142490372  Best Individual: [0, 9, 4, 7, 6, 1, 2, 8, 3, 5]
100
Generation 8 : Best Fittest: 0.3851091142490372  Best Individual: [0, 9, 4, 7, 6, 1, 2, 8, 3, 5]
100
Generation 9 : Best Fittest: 0.3851091142490372  Best Individual: [0, 9, 4, 7, 6, 1, 2, 8, 3, 5]
100
Generation 10 : Best Fittest: 0.3851091142490372  Best Individual: [0, 9, 4, 7, 6, 1, 2, 8,

In [283]:
current_time = time(10,0)
print('Starts at:', current_time)
count = 0
for place in route:
    i = POIs.index(place)
    if count >= 1:
        cost, current_time, arrival_time = waiting_cost(current_time, i-1, distance_time[j][i])
        if arrival_time>time(18,0):
            current_time = time(10,0)
        print('Arrive ',place, "at:", arrival_time)
        print('Leave ',place," at:", current_time)
        
#     else:
#         cost, current_time, arrival_time = waiting_cost(current_time, i)
#         print('Arrive ',place, " at:", arrival_time)
#         print('Leave ',place," at:", current_time)
    j = i
    count += 1


Starts at: 10:00:00
Arrive  Patan Durbar Square at: 10:18:48
Leave  Patan Durbar Square  at: 12:18:48
Arrive  Tribhuvan Museum at: 12:34:48
Leave  Tribhuvan Museum  at: 14:34:48
Arrive  Narayanhiti Palace at: 14:45:48
Leave  Narayanhiti Palace  at: 16:45:48
Arrive  Boudhanath (Stupa) at: 17:03:48
Leave  Boudhanath (Stupa)  at: 19:03:48
Arrive  Pashupatinath Temple at: 19:14:48
Leave  Pashupatinath Temple  at: 10:00:00
Arrive  Garden of Dreams at: 10:13:00
Leave  Garden of Dreams  at: 12:13:00
Arrive  Kasthamandap at: 12:22:00
Leave  Kasthamandap  at: 14:22:00
Arrive  Swayambhunath Stupa at: 14:34:00
Leave  Swayambhunath Stupa  at: 16:34:00
Arrive  Dakshinkali Temple at: 17:21:00
Leave  Dakshinkali Temple  at: 19:21:00


In [103]:
population

[[1, 6, 0, 4, 3, 9, 5, 8, 2, 7],
 [1, 6, 8, 4, 3, 9, 5, 0, 2, 7],
 [1, 6, 0, 4, 3, 9, 5, 8, 2, 7],
 [1, 6, 8, 4, 3, 9, 5, 0, 2, 7],
 [1, 6, 8, 4, 3, 9, 2, 0, 5, 7],
 [1, 6, 0, 4, 3, 9, 5, 8, 2, 7],
 [1, 6, 0, 4, 2, 5, 9, 8, 3, 7],
 [1, 6, 8, 4, 3, 9, 5, 0, 2, 7],
 [1, 6, 0, 4, 2, 5, 9, 8, 3, 7],
 [0, 1, 8, 4, 3, 9, 5, 6, 2, 7]]